<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# XGBoost

In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# Right now I am getting a useless warning every time I fit an `XGBoost` model.
# This line of code prevents warnings from being displayed. Not generally
# recommended.
warnings.filterwarnings(action='ignore')

In [3]:
%matplotlib inline

"Gradient boosting," like bagging, is a general method for training decision tree ensembles.

XGBoost ("eXtreme Gradient Boosting") is a particular implementation of gradient boosted decision trees. It is popular on Kaggle because it is both fast to train and often gives excellent predictive performance.

## Getting Started with XGBoost

We will use the `xgboost` library instead of scikit-learn for this lesson. Scikit-learn has [`GradientBoostingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) and [`GradientBoostingRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) classes, but they lack some of the tricks that have made the `xgboost` library popular.

`xgboost` provides estimators that use the same interface as scikit-learn's, so we will not need to change our approach.

In [4]:
# Import the xgboost package
# /scrub/
import xgboost as xgb

In [5]:
# Instantiate an XGBoost regressor
# /scrub/
xgb_reg = xgb.XGBRegressor()

**Exercise (20 mins., in pairs)**

- Load the Ames housing dataset from `assets/data/ames_train.csv` in this lesson's base directory.

In [6]:
# /scrub/
ames_df = pd.read_csv('../assets/data/ames_train.csv')

- Create a feature matrix DataFrame `X` containing all of the numeric columns from the Ames dataset except "Id" and the target column "SalePrice". Drop "OverallQual" to make things more interesting -- that very is very predictive but expensive to collect.

In [7]:
# /scrub/
target_col = 'SalePrice'

X = (ames_df
     .select_dtypes(['int64', 'float64'])
     .drop(['Id', 'OverallQual', target_col], axis='columns')
    )

- Create a target vector Series `y` with the values of the variable "SalePrice".

In [8]:
# /scrub/
y = ames_df.loc[:, target_col]

- Do a simple train/test split on `X` and `y`.

In [9]:
# /scrub/
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

- Fit an `XGBRegressor` on the training data.

In [10]:
# /scrub/
xgb_reg.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=100, n_jobs=0, num_parallel_tree=1,
       objective='reg:squarederror', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
       validate_parameters=1, verbosity=None)

- Get an R^2 score for the training set.

In [11]:
# /scrub/
xgb_reg.score(X_train, y_train)

0.999806045456122

- Get an R^2 score for the test set.

In [12]:
# /scrub/
xgb_reg.score(X_test, y_test)

0.8379362134973047

- Is your model overfitting, underfitting, both, or neither? How do you know?

/scrub/

Overfitting somewhat. The score on the training set is close to 1, so it does not seem to be underfitting, and the gap between training and test scores is moderately large.

- Load the Titanic dataset (located in this lesson's  `assets/data` directory).

In [13]:
# /scrub/
titanic = pd.read_csv('../assets/data/titanic.csv')

- Create a feature matrix `X` by dropping "PassengerId", "Name", "Ticket", and the target column "Survived." Dummy-code string columns as needed. Do NOT drop or impute missing values -- XGBoost handles them internally!

![](https://media.giphy.com/media/12NUbkX6p4xOO4/giphy.gif)

In [14]:
# /scrub/
X = titanic.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Survived'], axis='columns')

In [15]:
# /scrub/
X = pd.get_dummies(X, columns=['Sex', 'Embarked'])

- Create a target Series `y` with the values of "Survived."

In [16]:
# /scrub/
y = titanic.loc[:, 'Survived']

- Instantiate an `XGBClassifier`.

In [17]:
# /scrub/
xgb_clas = xgb.XGBClassifier()

- Fit and score the classifier on the entire dataset.

In [18]:
# /scrub/
xgb_clas.fit(X, y)
xgb_clas.score(X, y)

0.9708193041526375

- Get your classifier's score on held-out data using ten-fold cross-validation on the Titanic dataset, shuffling the rows before taking the folds.

In [19]:
# /scrub/
from sklearn.model_selection import cross_val_score, KFold

In [20]:
# /scrub/
kf = KFold(10, shuffle=True)

In [21]:
# /scrub/
scores = cross_val_score(xgb_clas, X, y, cv=kf)

In [22]:
# /scrub/
np.array(scores).mean()

0.8204369538077403

- Is your model overfitting, underfitting, both, or neither? How do you know?

/scrub/

Maybe neither, or slightly overfitting. The score on the training set seems pretty good, so it does not seem to be underfitting, and the gap between training and test scores is not very big.

$\blacksquare$

## XGBoost vs. Random Forests

### Similarities

Random forests and XGBoost both produce tree ensembles, and the provide many of the same parameters to reduce overfitting.

### Differences

#### Gradient Boosting vs. Bagging

- Bagging involves training each tree *independently* on a different *bootstrap sample*.
- Gradient boosting involves training each tree *sequentially to reduce the residual errors left by its predecessors*.

See [the official `xgboost` library documentation](https://xgboost.readthedocs.io/en/latest/tutorials/model.html) and Chapter 10 of [Elements of Statistical Learning](https://web.stanford.edu/~hastie/Papers/ESLII.pdf) for details.

#### Handling Missing Values

At each split for a given variable, XGBoost simply learns whether sending items with missing values left or right gives better results. This approach has a few advantages:

- It is automatic.
- Unlike dropping rows or columns, it allows you to use all of the values you do have.
- Unlike imputation, it treats "missing" as its own value rather than replacing it with some other value that might be wrong.

## Tuning XGBoost

`XGBoost` provides many options that you can tune to improve predictive performance.

### `n_estimators` and `learning_rate`

The `learning_rate` controls how "aggressive" each tree is in trying to correct the errors of its predecessors.

- If it is too low, then getting good predictive performance will require a large value for `n_estimators` (and thus a lot of time).
- If it is too high, then the algorithm will keep overshooting the target and won't coverge to good results.

Unlike with a random forest, setting `n_estimators` too high can hurt predictive performance with boosting because it leads to overfitting.

### Addressing Overfitting

#### Reducing Model Complexity

One way to address overfitting is to restrict model complexity more or less directly. `xgboost` provides many options for this purpose:

- Restricting tree shape
    - `max_depth` / `max_leaf_nodes` puts a hard limit on the depth or number of leaves in each tree
    - `gamma` is the minimum loss reduction required in order to make another split
    - `min_child_weight` is the minimum number of observations required in each child node in order to make a split, adjusted for the weight that is placed on each observation
- Restricting sizes of weights: `reg_lambda` and `reg_alpha` provide L1 and L2 regularization on sample weights, respectively

#### Adding Randomness

Another way to address overfitting when ensembling is to add randomness to the process of training each item in the ensemble.

- `subsample` specifies what proportion of the data is used to train each tree.
- `colsample_bytree` and `colsample_bylevel` specify what proportion of the features are available at the tree and split level, respectively.

### Example

We will use this general approach to tune our model:

- Find the optimal number of trees with default learning rate.
- Tune additional parameters.
- Lower learning rate and increase the number of trees.

#### Find Optimal Number of Trees with Default Learning Rate

In [23]:
# Split data by column
# /scrub/
target_col = 'SalePrice'

X = ames_df.select_dtypes(['int64', 'float64']).drop(['Id', 'OverallQual', target_col], axis='columns')
y = ames_df.loc[:, target_col]

In [24]:
# Instantiate model
# /scrub/
xgb_reg = xgb.XGBRegressor(learning_rate=0.1, max_depth=1)

In [25]:
# Fit and score on all data
# /scrub/
xgb_reg.fit(X, y)
print(xgb_reg.score(X, y))

0.8529772770319743


In [26]:
# Score with 5-fold CV
# /scrub/
kf = KFold(n_splits=5, shuffle=True, random_state=1)
np.array(cross_val_score(xgb_reg, X, y, cv=kf)).mean()

0.805096625229195

In [27]:
# Vary number of trees
# /scrub/
for num_trees in range(100, 1000, 100):
    xgb_reg = xgb.XGBRegressor(n_estimators=num_trees, max_depth=1)
    print(num_trees, np.array(cross_val_score(xgb_reg, X, y, cv=kf)).mean())

100 0.8366004966425535
200 0.8375681550240405
300 0.8359383418375476
400 0.8340299165071441
500 0.8322833691133817
600 0.831478123416369
700 0.8310230844903334
800 0.8302473936730767
900 0.8290506194525127


In [28]:
# Fix number of trees
# /scrub/
xgb_reg = xgb.XGBRegressor(n_estimators=400)

#### Tune Additional Parameters

scikit-learn has a `GridSearchCV` class that will run a model with various hyperparameter combinations and identify the combination that generated the best cross-validation scores.

In [29]:
# Try a few values for "max_depth" and "min_child_weight"
# /scrub/
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [1, 4, 7], 'min_child_weight': [1, 10, 100]
}
grid_search = GridSearchCV(
    xgb_reg,
    param_grid=param_grid,
    cv=kf
)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
       error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
       colsample_bynode=None, colsample_bytree=None, gamma=None,
       gpu_id=None, importance_type='gain', interaction_constraints=None,
       learning_rate=None, max_delta_step=None, max_depth=None,
       min_child_we..._pos_weight=None, subsample=None,
       tree_method=None, validate_parameters=None, verbosity=None),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': [1, 4, 7], 'min_child_weight': [1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [30]:
# Find out best parameters and their score
# /scrub/
grid_search.best_params_, grid_search.best_score_

({'max_depth': 1, 'min_child_weight': 1}, 0.8501283922179043)

In [31]:
# Try a few values for "subsample"
# /scrub/
param_grid = {
    'subsample': np.linspace(0.5, 1, 4)
}
xgb_reg = xgb.XGBRegressor(n_estimators=500, max_depth=4, min_child_weight=1)
grid_search = GridSearchCV(
    xgb_reg, 
    param_grid=param_grid,
    cv=4
)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
       colsample_bynode=None, colsample_bytree=None, gamma=None,
       gpu_id=None, importance_type='gain', interaction_constraints=None,
       learning_rate=None, max_delta_step=None, max_depth=4,
       min_child_weigh..._pos_weight=None, subsample=None,
       tree_method=None, validate_parameters=None, verbosity=None),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'subsample': array([0.5    , 0.66667, 0.83333, 1.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [32]:
# Find out best parameters and their score
# /scrub/
grid_search.best_params_, grid_search.best_score_

({'subsample': 1.0}, 0.8302317921178423)

In [33]:
# Get report on grid search results
# /scrub/
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,mean_train_score,std_train_score
0,0.630490,0.012615,0.007188,0.000998,0.5,{'subsample': 0.5},0.747250,0.829442,0.841482,0.759477,0.794444,0.041501,4,0.999998,0.999997,0.999987,0.999989,0.999993,0.000005
1,0.641308,0.006942,0.007460,0.000398,0.666667,{'subsample': 0.6666666666666666},0.745475,0.879521,0.822575,0.841460,0.822240,0.048864,2,0.999999,1.000000,0.999991,0.999990,0.999995,0.000004
2,0.685030,0.031581,0.007378,0.000605,0.833333,{'subsample': 0.8333333333333333},0.774624,0.810427,0.837766,0.799023,0.805466,0.022707,3,1.000000,1.000000,0.999992,0.999992,0.999996,0.000004
3,0.662147,0.008810,0.006888,0.000425,1,{'subsample': 1.0},0.794896,0.864909,0.844113,0.816961,0.830232,0.026561,1,0.999999,0.999999,0.999993,0.999993,0.999996,0.000003


The effect of one hyperparameter typically depends on the values of other hyperparameters -- for instance, increasing "max_depth" will have no effect if "min_child_weight" is sufficiently large. For this reason, it is generally valuable to do grid searches over multiple parameters simultaneously, rather than fixing one hyperparameter at a time. However, testing many combinations of many parameters can take a long time.

#### Lower Learning Rate and Increase the Number of Trees.

In [34]:
# Divide the learning rate by 10 and vary number of trees
# /scrub/
for num_trees in range(100, 3101, 500):
    xgb_reg = xgb.XGBRegressor(
        learning_rate=.01,
        n_estimators=num_trees,
        max_depth=4,
        min_child_weight=1,
        subsample=0.67,
    )
    print(num_trees, np.array(cross_val_score(xgb_reg, X, y, cv=kf)).mean())

100 -0.08719699741574949
600 0.8563178936418027
1100 0.8628480853282022
1600 0.864961902310833
2100 0.8656006970610834
2600 0.8662852078324466
3100 0.866567178314925


**Exercise (open-ended, in pairs)**

- Create the best XGBoost model you can for the Titanic dataset, as measured by accuracy in five-fold cross-validation. Use `GridSearchCV` at least once to search over at least two hyperparameters at a time.

$\blacksquare$

## Summary

- `XGBoost` is a popular decision tree ensemble algorithm.
- `XGBoost` uses gradient boosting, meaning that each tree attempts to correct the errors of previous trees.
- scikit-learn's `GridSearchCV` helps with testing hyperparameter values.